# Training a von Mises-Fisher SOMA model to label operating modes on EDF data

The task here is to train a von Mises-Fisher (vMF) SOMA model that can use continuous wind turbine (WT) measurementment features (e.g. power, wind speed, ...) to label different operating modes (OMs) and anomalies. The OMs are labels based on EDF WTOPeration (WTO) states, which serve as training and validation ground truth labels.

In this version, the 11 WTO labels known from the data are condensed into 4 basic sates, as definedbelow.

In [1]:
import numpy as np
import pandas as pd
import saga
import ran
from dask.distributed import Client

/opt/conda/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

The client can be either set up to work locally...

In [3]:
#client = Client(processes=True, n_workers=8, threads_per_worker=1)

... or to use cluster resources...

In [4]:
dask_uri = "10.18.95.45"  # get uri from running `make aws_scheduler_uri`
client = Client("{}:8786".format(dask_uri))

/opt/conda/lib/python3.7/site-packages/distributed/client.py:1078: VersionMismatchWarning: Mismatched versions found

toolz
+---------------------------+---------+
|                           | version |
+---------------------------+---------+
| client                    | 0.9.0   |
| scheduler                 | 0.10.0  |
| tcp://10.18.101.184:45067 | 0.10.0  |
| tcp://10.18.106.193:43685 | 0.10.0  |
| tcp://10.18.111.145:43877 | 0.10.0  |
| tcp://10.18.113.44:43083  | 0.10.0  |
| tcp://10.18.118.77:40445  | 0.10.0  |
| tcp://10.18.121.74:32865  | 0.10.0  |
| tcp://10.18.123.136:40671 | 0.10.0  |
| tcp://10.18.123.216:34195 | 0.10.0  |
| tcp://10.18.126.119:45587 | 0.10.0  |
| tcp://10.18.65.196:34915  | 0.10.0  |
| tcp://10.18.68.231:42353  | 0.10.0  |
| tcp://10.18.68.79:39647   | 0.10.0  |
| tcp://10.18.72.239:42427  | 0.10.0  |
| tcp://10.18.74.35:42735   | 0.10.0  |
| tcp://10.18.74.41:41747   | 0.10.0  |
| tcp://10.18.82.254:46053  | 0.10.0  |
| tcp://10.18.83.5:45649    | 0.10.0

## Rán and Ságá Preparation

In [5]:
manager = ran.load_manager_from_s3("s3://tagup-edf/models/ran_stage",
                                   client=client)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:root:
 Current Saga tables: ['sources', 'blocks', 'meta_data', 'tags', 'events', 'assets', 'sensor_bindings']


In [6]:
sb = manager.saga_backend_ext

## Features and Labels

In [7]:
# synchronous features used in TTE
SYNC_FEAT =  ['active_power_avg_sql_10_min_avg',
              'amb_wind_speed_avg_sql_10_min_avg',
              'amb_wind_speed_1_avg_sql_10_min_avg',
              'amb_wind_speed_2_avg_sql_10_min_avg',
              'ambient_temp_avg_sql_10_min_avg',
              'blds_blade_a_pitch_angle_avg_sql_10_min_avg',
              #'blds_blade_a_pitch_angle_ref_avg_sql_10_min_avg',
              'blds_blade_b_pitch_angle_avg_sql_10_min_avg',
              #'blds_blade_b_pitch_angle_ref_avg_sql_10_min_avg',
              'blds_blade_c_pitch_angle_avg_sql_10_min_avg',
              #'blds_blade_c_pitch_angle_ref_avg_sql_10_min_avg',
              'current_l_1_avg_sql_10_min_avg',
              'current_l_2_avg_sql_10_min_avg',
              'current_l_3_avg_sql_10_min_avg',
              'frequency_avg_sql_10_min_avg',
              'gen_rpm_avg_sql_10_min_avg',
              'nacelle_temp_avg_sql_10_min_avg',
              'power_factor_sql_10_min_avg',
              're_active_power_avg_sql_10_min_avg',
              'rtr_rpm_avg_sql_10_min_avg',
              #'tower_humidity_avg_sql_10_min_avg',   # this seems to cause limited models, dominating?
              'voltage_l_1_avg_sql_10_min_avg',
              'voltage_l_2_avg_sql_10_min_avg',
              'voltage_l_3_avg_sql_10_min_avg',
              'windspeed_gust_max_total_sql_10_min_avg',
              'windspeed_mean_total_sql_10_min_avg',
              'yaw_position_avg_sql_10_min_avg',
             ]
# other features used in TTE:
ERROR = [     'error_code_13110', 
              'error_code_13902', 
              'error_code_14214', 
              'error_code_15324', 
              'error_code_emergency_switch_activated', 
              'error_code_inv._cool._w._temp_high',  
              'error_code_inv._cooling_water_temp._warn.', 
              'error_code_t._o._r_converter_coolant_heater', 
              'error_code_t._o._r_converter_coolant_pump', 
              'error_code_t._o._r_converter_fan_(_tower)', 
              'error_code_tor_transfor._ventilation',   
              'error_code_tower_emergency_contact._err',              
             ]
STATUS =     ['wps_status_fault_condition',
              'wps_status_no_communication_with_wtc',
              'wps_status_service_mode'
             ]
OPERATION =  ['wtoperation_state_cable_untwisting_procedure_active',
              'wtoperation_state_pitchlubrication_procedure_active',
              'wtoperation_state_stopped_due_to_grid_error',
              'wtoperation_state_stopped_due_to_high_wind',
              'wtoperation_state_waiting_for_sufficient_wind_speed',
              'wtoperation_state_remote_stop_command_is_active',
              'wtoperation_state_stopped_due_to_internal_error._autoreset_when_error_state_has_gone',
              'wtoperation_state_stopped_due_to_internal_error._manual_reset_is_required.',
             ]
OTHER    =   ['hyd_pressure', 
              'trafo_room_temp',
              'trafo_temp',
             ]

query Saga for all available features from one WT:

In [8]:
all_feat = sb.get_blocks_for(customer_name="edf", asset_name="teesside_wt_01")["measurement"].values

In [9]:
FEAT = SYNC_FEAT + OTHER

In [10]:
print("Chosen {} of {} features.".format(len(FEAT), len(all_feat)))

Chosen 26 of 441 features.


### Labels:
List all known WTO modes.

In [11]:
WTO = [
    'wt_operation_state_stopped_due_to_internal_error_manual_reset_is_required',
    'wt_operation_state_stopped_due_to_internal_error_autoreset_when_error_state_has_gone',
    'wt_operation_state_remote_stop_command_is_active',
    'wt_operation_state_cable_untwisting_procedure_active',
    'wt_operation_state_pitchlubrication_procedure_active',
    'wt_operation_state_ready_for_production_waiting_for_release_operation_command',
    'wt_operation_state_stopped_due_to_high_wind',
    'wt_operation_state_stopped_due_to_grid_error',
    'wt_operation_state_waiting_for_sufficient_wind_speed',
    #'wt_operation_state_intf_shut',       # position?
    #'wt_operation_state_n_transitions',   # position?
    'wt_operation_state_in_operation_waiting_for_grid',
    'wt_operation_state_in_operation_generator_on_grid',
    ]

### Simplified WTO labels: 
Group above labels under fewer group names.

In [12]:
simplify_wto = {
    'wt_operation_state_stopped_due_to_internal_error_manual_reset_is_required' : 
        'stopped due to turbine error',
    'wt_operation_state_stopped_due_to_internal_error_autoreset_when_error_state_has_gone':
        'stopped due to turbine error',
    'wt_operation_state_remote_stop_command_is_active':
        'stopped due to turbine error',
    'wt_operation_state_cable_untwisting_procedure_active':
        'in regular maintenance mode',
    'wt_operation_state_pitchlubrication_procedure_active':
        'in regular maintenance mode',
    'wt_operation_state_ready_for_production_waiting_for_release_operation_command':
        'in regular maintenance mode',
    'wt_operation_state_stopped_due_to_high_wind':
        'stopped due to external error',
    'wt_operation_state_stopped_due_to_grid_error':
        'stopped due to external error',
    'wt_operation_state_waiting_for_sufficient_wind_speed':
        'stopped due to external error',
    #'wt_operation_state_intf_shut',       # position?
    #'wt_operation_state_n_transitions',   # position?
    'wt_operation_state_in_operation_waiting_for_grid':
        'in operation',
    'wt_operation_state_in_operation_generator_on_grid':
        'in operation',
}

In [13]:
wto_label_map = list(set(simplify_wto.values()))

In [14]:
wto_label_map

['in regular maintenance mode',
 'stopped due to turbine error',
 'stopped due to external error',
 'in operation']

The order of the above list is important. One time stamp can have multiple labels, and the order of this list will determine which label overrules the other in that case. Labels listed earlier in this list will take precedence over those following after them. (This happens by choosing the first entry in the `fs` function seen below.)

## Datasets

This function queries Saga and composes a so-called data_spec, listing all the Saga blocks that will be used as input data for each asset. 

In [15]:
def compose_dataset(wt_ids, sb, start="2016-01-01", end="2016-12-31", features=FEAT, oms=WTO):
    """
    Compose dataset for each asset ID, creating a list of dicts with a Saga data-spec entries for data and labels.
    
    Arguments:
        wt_ids: list of Saga asset IDs for the WTs.
        sb: Saga backend reference
        start: start timestamp for data selection
        end: end timestamp for data selection
        features: measurement names in Saga to be selected
        oms: OM column names, being measurement names in Saga since each 
             WT OM has its own column, filled with booleans.
             
    Returns:
        A Ran dataset, which is a list of samples, i.e. dictionaries that map Saga data_specs to 
        keywords such as 'data' and 'wt_operation' for each asset.
    """
    ds = []
    for i in wt_ids:
        ds.append({
            "asset_id": "teesside_wt_{:02d}".format(i),
            "data": sb.get_blocks_for(
                asset_name=["teesside_wt_{:02d}".format(i)],
                measurement=features,
                tag_name=["alex"],
                start=start,
                end=end,
                return_as='specification'
            ),
            "wt_operation": sb.get_blocks_for(
                asset_name=["teesside_wt_{:02d}".format(i)],
                measurement=oms,
                tag_name=["alex"],
                start=start,
                end=end,
                return_as='specification'
            )
        })
    return ds

### Preprocessing

This is setting up all the preprocessing steps required to convert the Saga data_specs into the data input for the SOMA algorithm.   
See here for more information [Ran Preprocessing](https://tagup.atlassian.net/wiki/spaces/FRIGG/pages/25971260833/User+Specifications#Preprocessing).

In [16]:
# merge OMs and choose first (see order above).
fs = ("lambda x: np.nan if (np.sum(x[{}]) == 0) else "
      "np.array({})[list(x[{}])][0]".format(
              WTO, WTO, WTO))

prep = [
        {'type': 'ReadSagaSpec',
         'input': {'data': 'data'},
         },
        {'type': 'ReadSagaSpec',
         'input': {'data': 'wt_operation'},
         },
        {'type': 'Resample',
         'input': {'data': 'data'},
         'params': {
             'interval': '10min', #'1H',
             'aggregator': 'mean'}
         },
        {'type': 'Resample',
         'input': {'data': 'wt_operation'},
         'params': {
             'interval': '10min', #'1H',
             'aggregator': 'max'}
         },
        {'type': 'ApplyPerRow',
         'input': {'data': 'wt_operation'},
         'params': {
             'col': 'wto',
             'func': fs}
         },
        {'type': 'Replace',
         'input': {'data': 'wt_operation'},
         'params': {
             'to_replace': simplify_wto}
         },
        {'type': 'SelectColumns',
         'input': {'data': 'wt_operation'},
         'output': {'data': 'labels'},
         'params': {
             'selection': 'wto'}
         }
]


# Training

In [17]:
def train_vmf(train_data, val_data, preprocessing, cc, rff_m, label_map=None):
    """
    Create a Ran specification for training and validating a von Mises-Fisher (vMF) model.
    
    Arguments:
        train_data: Ran dataset for training, i.e. list of asset samples that contain Saga data_specs.
        val_data: Ran dataset for validation, i.e. list of asset samples that contain Saga data_specs.
        preprocessing: Ran preprocessing stages.
        cc: cluster count, vMF hyperparameter for internal/hidden number of sub-clusters.
        rff_m: Random Fourier Features count, defining the vMF input data transformation
        label_map: List of label names that appear in the data; required for distributed execution.
        
    Returns:
        A Ran specification.
    """
    spec = {
        'problem_type': 'soma',
        'description': 'EDF: vMF clustering for WTOperation states with 27 turbines, 25 features and 11 operation labels.',
        'preprocessing': preprocessing,
        'training': {
            'model_type': 'RFFNormalizedFeatureModel',
            'parameters': {
                'distributed': True,
                'random_state': 42,
                'cluster_count': cc,
                'rff_m': rff_m,
                'label_map': label_map
            }
        },
        'experiment': {
            'preprocessing': [                
                {
                    'type': 'DropNaNs',
                    'input': {'data': 'labels'}
                },
                {
                    'type': 'SelectRowsByOtherIndex',
                    'input': {'data': 'data',
                              'other': 'labels'}
                }
            ],
            'validation': {
                'metrics': {
                    'class_scores': "class_indiv_scores"
                }
            }
        },
    }
    if isinstance(train_data, str):
        spec["training"]["dataset_id"] = train_data
    else:
        spec["training"]["dataset"] = train_data
        
    if isinstance(val_data, str):
        spec["experiment"]["validation"]["dataset_id"] = val_data
    else:
        spec["experiment"]["validation"]["dataset"] = val_data
    
    return spec

#### For cluster: A dataset including all turbines, training on 2016 and validating on 2017 data

In [18]:
t_1t27_dataset = compose_dataset(np.arange(1, 28), sb, "2016-01-01", "2016-12-31", FEAT, WTO)
# 11 labeled states
v_1t27_dataset = compose_dataset(np.arange(1, 28), sb, "2017-01-01", "2017-07-31", FEAT, WTO)


In [19]:
print("Training dataset has {} samples, each containing saga data_specs with {} features "
      "and {} operation labels.".format(len(t_1t27_dataset),
                                        len(t_1t27_dataset[0]['data']['block_ids']),
                                        len(t_1t27_dataset[0]['wt_operation']['block_ids'])))

Training dataset has 27 samples, each containing saga data_specs with 26 features and 11 operation labels.


### Train and validate

In [21]:
mids = []
eids = [] 

In [20]:
for cc in [12, 20, 50, 100]:
    for rffm in [150, 250]:  
        print("\n Training vMF with {} clusters and {} RFF.".format(cc, rffm))
        m, e = manager.run_specification(train_vmf(t_1t27_dataset, v_1t27_dataset, prep, cc, rffm, wto_label_map))
        mids.append(m)
        eids.append(e)

INFO:root:Training model of problem type soma



 Training vMF with 12 clusters and 150 RFF.


INFO:root:Stored dataset 0eceb38a7172839edcdcad073e77af41d6dcc0b5.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace', 'SelectColumns']
INFO:root:Started training RFFNormalizedFeatureModel on 27 workers.
INFO:root:Completed training routine for RFFNormalizedFeatureModel, storing model.
INFO:root:Successfully trained soma model: 8719e628348266239b57057340038917f9a49987.

INFO:root:Found validation specification, loading dataset.
INFO:root:Stored dataset c16f3d6880a51605c4ec13cf06ed3cc87f42c9a8.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace


 Training vMF with 12 clusters and 250 RFF.


INFO:root:Dataset with id 0eceb38a7172839edcdcad073e77af41d6dcc0b5 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace', 'SelectColumns']
INFO:root:Started training RFFNormalizedFeatureModel on 27 workers.
INFO:root:Completed training routine for RFFNormalizedFeatureModel, storing model.
INFO:root:Successfully trained soma model: 5a6ef578ac38122646aa9fdb446b629eba9e8607.

INFO:root:Found validation specification, loading dataset.
INFO:root:Dataset with id c16f3d6880a51605c4ec13cf06ed3cc87f42c9a8 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
IN

...

INFO:root:Dataset with id 0eceb38a7172839edcdcad073e77af41d6dcc0b5 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace', 'SelectColumns']
INFO:root:Started training RFFNormalizedFeatureModel on 27 workers.
INFO:root:Completed training routine for RFFNormalizedFeatureModel, storing model.
INFO:root:Successfully trained soma model: 2f8d6d39f351856710cebeab3e025f601c88151b.

INFO:root:Found validation specification, loading dataset.
INFO:root:Dataset with id c16f3d6880a51605c4ec13cf06ed3cc87f42c9a8 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
IN


 Training vMF with 100 clusters and 150 RFF.


INFO:root:Dataset with id 0eceb38a7172839edcdcad073e77af41d6dcc0b5 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace', 'SelectColumns']
INFO:root:Started training RFFNormalizedFeatureModel on 27 workers.
INFO:root:Completed training routine for RFFNormalizedFeatureModel, storing model.
INFO:root:Successfully trained soma model: 169ccada3371389e1e7f92d10b9b6e5d78d1d46a.

INFO:root:Found validation specification, loading dataset.
INFO:root:Dataset with id c16f3d6880a51605c4ec13cf06ed3cc87f42c9a8 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
IN


 Training vMF with 100 clusters and 250 RFF.


INFO:root:Dataset with id 0eceb38a7172839edcdcad073e77af41d6dcc0b5 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace', 'SelectColumns']
INFO:root:Started training RFFNormalizedFeatureModel on 27 workers.
INFO:root:Completed training routine for RFFNormalizedFeatureModel, storing model.
INFO:root:Successfully trained soma model: 28250f98449d33e962898b3a91be7e6c667250b2.

INFO:root:Found validation specification, loading dataset.
INFO:root:Dataset with id c16f3d6880a51605c4ec13cf06ed3cc87f42c9a8 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
IN

### Review Validation Results:

In [21]:
for m, e in zip (mids, eids):
    print("Model {}".format(m))
    print(manager.get_model_spec(m)["training"]["parameters"])
    display(manager.get_experiment_validation(e)['class_scores'])
    print()

Model 8719e628348266239b57057340038917f9a49987
{'distributed': True, 'random_state': 42, 'cluster_count': 12, 'rff_m': 150, 'label_map': ['in operation', 'stopped due to turbine error', 'stopped due to external error', 'in regular maintenance mode']}


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.000000,NaN,0,5184
in operation,0.769727,0.912018,66764,79106
in regular maintenance mode,NaN,0.000000,1528,0
stopped due to external error,0.815536,0.192638,18039,4261
stopped due to turbine error,NaN,0.000000,2220,0



Model 5a6ef578ac38122646aa9fdb446b629eba9e8607
{'distributed': True, 'random_state': 42, 'cluster_count': 12, 'rff_m': 250, 'label_map': ['in operation', 'stopped due to turbine error', 'stopped due to external error', 'in regular maintenance mode']}


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.000000,NaN,0,5879
in operation,0.767184,0.904410,66764,78706
in regular maintenance mode,NaN,0.000000,1528,0
stopped due to external error,0.831316,0.182771,18039,3966
stopped due to turbine error,NaN,0.000000,2220,0



Model d7daa3af618a1f3086ed972640ad2ac36aaabb20
{'distributed': True, 'random_state': 42, 'cluster_count': 20, 'rff_m': 150, 'label_map': ['in operation', 'stopped due to turbine error', 'stopped due to external error', 'in regular maintenance mode']}


...


Model 169ccada3371389e1e7f92d10b9b6e5d78d1d46a
{'distributed': True, 'random_state': 42, 'cluster_count': 100, 'rff_m': 150, 'label_map': ['in operation', 'stopped due to turbine error', 'stopped due to external error', 'in regular maintenance mode']}


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.000000,NaN,0,200
in operation,0.834010,0.920241,66764,73667
in regular maintenance mode,NaN,0.000000,1528,0
stopped due to external error,0.622631,0.504518,18039,14617
stopped due to turbine error,0.895522,0.027027,2220,67



Model 28250f98449d33e962898b3a91be7e6c667250b2
{'distributed': True, 'random_state': 42, 'cluster_count': 100, 'rff_m': 250, 'label_map': ['in operation', 'stopped due to turbine error', 'stopped due to external error', 'in regular maintenance mode']}


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.000000,NaN,0,1786
in operation,0.816632,0.937077,66764,76611
in regular maintenance mode,NaN,0.000000,1528,0
stopped due to external error,0.761542,0.420644,18039,9964
stopped due to turbine error,0.368421,0.031532,2220,190


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR:asyncio:_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


A few more:

In [22]:
for rffm in [250, 300]:  
    for cc in [100, 150, 200]:
        print("\n Training vMF with {} clusters and {} RFF.".format(cc, rffm))
        m, e = manager.run_specification(train_vmf(t_1t27_dataset, v_1t27_dataset, prep, cc, rffm, wto_label_map))
        mids.append(m)
        eids.append(e)

INFO:root:Training model of problem type soma



 Training vMF with 100 clusters and 250 RFF.


INFO:root:Dataset with id 2b80f26eabf43ef43b1e87d4d4db0be0502faa10 already exists in this index, no new dataset was saved.
INFO:root:A model with the same parameters and description already exists in this index (id 8b827ec4c4eb8d360c157d47e1fecf6413adf63f), no training executed.
INFO:root:Training model of problem type soma



 Training vMF with 150 clusters and 250 RFF.


INFO:root:Dataset with id 2b80f26eabf43ef43b1e87d4d4db0be0502faa10 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace', 'SelectColumns']
INFO:root:Started training RFFNormalizedFeatureModel on 27 workers.
INFO:root:Completed training routine for RFFNormalizedFeatureModel, storing model.
INFO:root:Successfully trained soma model: 0d4b3a07c10ef7028960e7336b82a3b70f3db97d.

INFO:root:Found validation specification, loading dataset.
INFO:root:Dataset with id 7be51dfd4e4a2cb0c232d95166a120b6fcd6e136 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
IN


 Training vMF with 200 clusters and 250 RFF.


INFO:root:Dataset with id 2b80f26eabf43ef43b1e87d4d4db0be0502faa10 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace', 'SelectColumns']
INFO:root:Started training RFFNormalizedFeatureModel on 27 workers.
INFO:root:Completed training routine for RFFNormalizedFeatureModel, storing model.
INFO:root:Successfully trained soma model: 9ff2c03b3e244aff3a9cc27e90d56a15ac33486d.

INFO:root:Found validation specification, loading dataset.
INFO:root:Dataset with id 7be51dfd4e4a2cb0c232d95166a120b6fcd6e136 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
IN


 Training vMF with 100 clusters and 300 RFF.


INFO:root:Dataset with id 2b80f26eabf43ef43b1e87d4d4db0be0502faa10 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace', 'SelectColumns']
INFO:root:Started training RFFNormalizedFeatureModel on 27 workers.


AssertionError: (array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
       -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
       -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
       -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
       -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
       -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
       -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
       -inf, -inf, -inf, -inf, -

In [31]:
leids = manager.get_experiment_ids_by_dates("20210204", "20210205")
lmids = []
for e in leids:
    m = manager.experiment_index.loc[e, "model_id"]
    lmids.append(m)
    print("Model {}".format(m))
    print("  training time: {:6.1F} s".format(manager.model_index.loc[m, "stats"]["training_time"]))
    mpars = manager.get_model_spec(m)["training"]["parameters"]
    print("  Cluster count: {:3d}".format(mpars["cluster_count"]))
    print("  RFF count:     {:3d}".format(mpars["rff_m"]))
    display(manager.get_experiment_validation(e)['class_scores'].round(2))
    print()

Model 2b4e43e460dd6a2a821e5a904884c009a61a04e3
  training time:  277.8 s
  Cluster count:  20
  RFF count:     150


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.00,NaN,0,4532
in operation,0.74,0.78,19593,20555
in regular maintenance mode,NaN,0.00,474,0
stopped due to external error,0.73,0.22,5705,1722
stopped due to turbine error,NaN,0.00,1037,0



Model ebc052ba3f32ae2d60468e286b54346e94e03690
  training time:  235.0 s
  Cluster count:  20
  RFF count:     250


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.00,NaN,0,2468
in operation,0.76,0.85,19593,21736
in regular maintenance mode,NaN,0.00,474,0
stopped due to external error,0.77,0.35,5705,2605
stopped due to turbine error,NaN,0.00,1037,0



Model 8719e628348266239b57057340038917f9a49987
  training time:  144.2 s
  Cluster count:  12
  RFF count:     150


...

,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.00,NaN,0,1786
in operation,0.82,0.94,66764,76611
in regular maintenance mode,NaN,0.00,1528,0
stopped due to external error,0.76,0.42,18039,9964
stopped due to turbine error,0.37,0.03,2220,190



Model 0d4b3a07c10ef7028960e7336b82a3b70f3db97d
  training time:  543.9 s
  Cluster count: 150
  RFF count:     250


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.00,NaN,0,9
in operation,0.84,0.96,66764,76442
in regular maintenance mode,NaN,0.00,1528,0
stopped due to external error,0.75,0.50,18039,11957
stopped due to turbine error,0.95,0.06,2220,143



Model 9ff2c03b3e244aff3a9cc27e90d56a15ac33486d
  training time:  903.3 s
  Cluster count: 200
  RFF count:     250


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.00,NaN,0,807
in operation,0.84,0.95,66764,75107
in regular maintenance mode,NaN,0.00,1528,0
stopped due to external error,0.76,0.49,18039,11764
stopped due to turbine error,0.08,0.03,2220,873


In [32]:
for rffm in [200]:  
    for cc in [100, 150, 200]:
        print("\n Training vMF with {} clusters and {} RFF.".format(cc, rffm))
        m, e = manager.run_specification(train_vmf(t_1t27_dataset, v_1t27_dataset, prep, cc, rffm, wto_label_map))
        mids.append(m)
        eids.append(e)

INFO:root:Training model of problem type soma



 Training vMF with 100 clusters and 200 RFF.


INFO:root:Dataset with id 2b80f26eabf43ef43b1e87d4d4db0be0502faa10 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace', 'SelectColumns']
INFO:root:Started training RFFNormalizedFeatureModel on 27 workers.
INFO:root:Completed training routine for RFFNormalizedFeatureModel, storing model.
INFO:root:Successfully trained soma model: 3e86319736c51565c2137eebfa5fd392d569e7b0.

INFO:root:Found validation specification, loading dataset.
INFO:root:Dataset with id 7be51dfd4e4a2cb0c232d95166a120b6fcd6e136 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
IN


 Training vMF with 150 clusters and 200 RFF.


INFO:root:Dataset with id 2b80f26eabf43ef43b1e87d4d4db0be0502faa10 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace', 'SelectColumns']
INFO:root:Started training RFFNormalizedFeatureModel on 27 workers.
INFO:root:Completed training routine for RFFNormalizedFeatureModel, storing model.
INFO:root:Successfully trained soma model: 7c5076ad81d2a898bc7b0b30d22cfa327b416132.

INFO:root:Found validation specification, loading dataset.
INFO:root:Dataset with id 7be51dfd4e4a2cb0c232d95166a120b6fcd6e136 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
IN


 Training vMF with 200 clusters and 200 RFF.


INFO:root:Dataset with id 2b80f26eabf43ef43b1e87d4d4db0be0502faa10 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
INFO:root:  1. chain, per sample: ['ReadSagaSpec', 'ReadSagaSpec', 'Resample', 'Resample', 'ApplyPerRow', 'Replace', 'SelectColumns']
INFO:root:Started training RFFNormalizedFeatureModel on 27 workers.
INFO:root:Completed training routine for RFFNormalizedFeatureModel, storing model.
INFO:root:Successfully trained soma model: 2286d2919b9a4d4bff24bf8d700c5e9db07e08d1.

INFO:root:Found validation specification, loading dataset.
INFO:root:Dataset with id 7be51dfd4e4a2cb0c232d95166a120b6fcd6e136 already exists in this index, no new dataset was saved.
INFO:root:Composed PreprocessingChain(s) (grouping of stages that run distributedly, i.e. per sample, vs those requiring all samples, i.e. the merged dataset): 
IN

In [33]:
leids = manager.get_experiment_ids_by_dates("20210205", "20210205")
lmids = []
for e in leids:
    m = manager.experiment_index.loc[e, "model_id"]
    lmids.append(m)
    print("Model {}".format(m))
    print("  training time: {:6.1F} s".format(manager.model_index.loc[m, "stats"]["training_time"]))
    mpars = manager.get_model_spec(m)["training"]["parameters"]
    print("  Cluster count: {:3d}".format(mpars["cluster_count"]))
    print("  RFF count:     {:3d}".format(mpars["rff_m"]))
    display(manager.get_experiment_validation(e)['class_scores'].round(2))
    print()

Model 8b827ec4c4eb8d360c157d47e1fecf6413adf63f
  training time:  518.5 s
  Cluster count: 100
  RFF count:     250


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.00,NaN,0,1786
in operation,0.82,0.94,66764,76611
in regular maintenance mode,NaN,0.00,1528,0
stopped due to external error,0.76,0.42,18039,9964
stopped due to turbine error,0.37,0.03,2220,190



Model 0d4b3a07c10ef7028960e7336b82a3b70f3db97d
  training time:  543.9 s
  Cluster count: 150
  RFF count:     250


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.00,NaN,0,9
in operation,0.84,0.96,66764,76442
in regular maintenance mode,NaN,0.00,1528,0
stopped due to external error,0.75,0.50,18039,11957
stopped due to turbine error,0.95,0.06,2220,143



Model 9ff2c03b3e244aff3a9cc27e90d56a15ac33486d
  training time:  903.3 s
  Cluster count: 200
  RFF count:     250


...

,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.00,NaN,0,1358
in operation,0.84,0.94,66764,74103
in regular maintenance mode,NaN,0.00,1528,0
stopped due to external error,0.68,0.47,18039,12569
stopped due to turbine error,0.15,0.04,2220,521



Model 7c5076ad81d2a898bc7b0b30d22cfa327b416132
  training time:  614.2 s
  Cluster count: 150
  RFF count:     200


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.00,NaN,0,1326
in operation,0.83,0.90,66764,71640
in regular maintenance mode,NaN,0.00,1528,0
stopped due to external error,0.57,0.46,18039,14686
stopped due to turbine error,0.14,0.06,2220,899



Model 2286d2919b9a4d4bff24bf8d700c5e9db07e08d1
  training time:  615.1 s
  Cluster count: 200
  RFF count:     200


,precision,recall,true_count,pred_count
Missing Data,NaN,NaN,0,0
Anomaly,0.00,NaN,0,995
in operation,0.86,0.92,66764,71459
in regular maintenance mode,NaN,0.00,1528,0
stopped due to external error,0.62,0.53,18039,15209
stopped due to turbine error,0.15,0.06,2220,888
